In [1]:
import math
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from perceptronac.convex_hull import convex_hull

In [7]:
def points_in_convex_hull(fname):

    data = pd.read_csv(fname,index_col="topology")
    normalized_data = data.copy()
    normalized_data["complexity"] = normalized_data["complexity"].apply(lambda x : math.log10(x)) # complexity in logarithmic scale
    normalized_data = MinMaxScaler().fit_transform(normalized_data.values) # make the coordinates in the range [0,1]

    plt.plot(normalized_data[:,0],normalized_data[:,1],linestyle="",marker="x") # verify that everything is alright

    chull = normalized_data[convex_hull(normalized_data.tolist()),:]
    plt.plot(normalized_data[:,0],normalized_data[:,1],linestyle="",marker="x")
    plt.plot(chull[:,0],chull[:,1],linestyle="solid",color="red",marker=None)

    plt.show()

    return data.iloc[convex_hull(normalized_data.tolist()),:]

In [1]:

points_in_convex_hull("/home/lucas/Documents/perceptronac/results/exp_1656174158/exp_1656174158_train_values.csv")


In [2]:
points_in_convex_hull("/home/lucas/Documents/perceptronac/results/exp_1656174158/exp_1656174158_valid_values.csv")

In [55]:
import torch
from perceptronac.models import ArbitraryMLP
import pandas as pd
import ast
import numpy as np
import matplotlib.pyplot as plt

In [36]:
configs = pd.read_csv("/home/lucas/Documents/perceptronac/results/exp_1656174158/exp_1656174158_conf.csv",index_col="key").to_dict()["value"]
configs['save_dir'] = "/home/lucas/Documents/perceptronac/results/"
topologies = ast.literal_eval(configs["topologies"])

In [37]:
widths = topologies[0]
file_name = f"{configs['save_dir'].rstrip('/')}/exp_{configs['id']}/exp_{configs['id']}_{'_'.join(map(str,widths))}_min_valid_loss_model.pt"

In [38]:
model = ArbitraryMLP(widths)
model.load_state_dict(torch.load(file_name))

<All keys matched successfully>

In [50]:
all_parameters = []
for i in range(len(model.layers)):
    if isinstance(model.layers[i], torch.nn.Linear):
        with torch.no_grad():
            all_parameters.append(model.layers[i].weight.data.detach().numpy().reshape(-1))
            all_parameters.append(model.layers[i].bias.data.detach().numpy().reshape(-1))
all_parameters = np.concatenate(all_parameters,axis=0)

In [61]:
max_abs_value = np.max(np.abs(all_parameters))

In [210]:
y,x = np.histogram(all_parameters,np.linspace(-max_abs_value,max_abs_value,100))
plt.plot(x[:-1],y)

In [69]:
n_bits = 8
Delta = (max_abs_value - (-max_abs_value))/(2**n_bits)

In [211]:
np.round(all_parameters/Delta)

In [212]:
Delta * np.round(all_parameters/Delta)

In [79]:
for i in range(len(model.layers)):
    if isinstance(model.layers[i], torch.nn.Linear):
        with torch.no_grad():
            model.layers[i].weight.data = Delta * torch.round(model.layers[i].weight.data/Delta)
            model.layers[i].bias.data = Delta * torch.round(model.layers[i].bias.data/Delta)

In [83]:
all_parameters2 = []
for i in range(len(model.layers)):
    if isinstance(model.layers[i], torch.nn.Linear):
        with torch.no_grad():
            all_parameters2.append(model.layers[i].weight.data.detach().numpy().reshape(-1))
            all_parameters2.append(model.layers[i].bias.data.detach().numpy().reshape(-1))
all_parameters2 = np.concatenate(all_parameters2,axis=0)

In [84]:
np.allclose( all_parameters2, Delta * np.round(all_parameters/Delta) )

True

In [186]:
quantized = np.round( all_parameters2/Delta ).astype(int)

In [200]:
def int_to_bits(v):
    sign = 1 if v < 0 else 0
    bitseq = [int(c) for c in "{:08b}".format(np.abs(v))]
    bitseq[0] = sign
    return bitseq

In [201]:
bitstream = np.array([int_to_bits(v) for v in np.round( all_parameters2/Delta ).astype(int)]).reshape(-1)

In [202]:
from perceptronac.perfect_AC import perfect_AC_binary
from perceptronac.models import StaticAC

staticac = StaticAC()

In [203]:
staticac.load(bitstream.reshape(-1,1))

In [204]:
perfect_AC_binary(bitstream.reshape(-1,1),staticac(bitstream.reshape(-1,1)))

0.9631314941179526

In [208]:
def entropy(seq):
    p = np.zeros(len(np.unique(seq)))
    for i,v in enumerate(np.unique(seq)):
        p[i] = np.sum(seq == v)/len(seq)
    # p = np.histogram(quantized.flatten(), bins=np.arange(quantized.min(), quantized.max()+1))[0]
    # p = np.delete(p, p==0)
    # p = p / len(seq)
    return -(p * np.log2(p)).sum()

In [209]:
entropy(quantized)

6.636958446600158

In [205]:
0.9631314941179526 * 8

7.705051952943621